In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os
import imutils

In [2]:
prototxtPath=os.path.sep.join([r'C:\Yash\College\SEM 6\SDP\Project 3.0','deploy.prototxt'])
weightsPath=os.path.sep.join([r'C:\Yash\College\SEM 6\SDP\Project 3.0','res10_300x300_ssd_iter_140000.caffemodel'])

In [3]:
prototxtPath

'C:\\Yash\\College\\SEM 6\\SDP\\Project 3.0\\deploy.prototxt'

In [4]:
net=cv2.dnn.readNet(prototxtPath,weightsPath)

In [5]:
model=load_model(r'C:\Yash\College\SEM 6\SDP\Project 3.0\mobilenet_v2_2.model')

In [16]:
image=cv2.imread(r"C:\Yash\College\SEM 6\SDP\Project 3.0\GettyImages_1198631333.0.jpg",1)

In [17]:
(h,w)=image.shape[:2]

In [18]:
(h,w)

(800, 1200)

In [19]:
blob=cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))

In [20]:
blob

array([[[[140., 127., 134., ...,  39.,  43.,  45.],
         [101., 133., 138., ...,  32.,  35.,  35.],
         [137.,  38.,  54., ...,  46.,  47.,  47.],
         ...,
         [-60., -63., -65., ...,  -8.,   2.,  13.],
         [-64., -56., -67., ...,  -7., -14.,  32.],
         [-54., -58., -77., ...,  -6., -16.,  12.]],

        [[ 63.,  47.,  54., ...,  -4.,  -1.,  -1.],
         [ 17.,  49.,  54., ...,   2.,   3.,   1.],
         [ 48., -51., -34., ...,   1.,  -1.,  -2.],
         ...,
         [ -8., -38., -12., ..., -99., -76., -68.],
         [-21., -27.,  -4., ..., -94., -91., -51.],
         [-39.,  -9.,  11., ..., -90., -93., -73.]],

        [[108.,  92.,  99., ...,  51.,  54.,  54.],
         [ 63.,  94., 100., ...,  58.,  59.,  58.],
         [ 95.,  -4.,  13., ...,  60.,  59.,  58.],
         ...,
         [ 41.,  14.,  32., ..., -34., -21., -16.],
         [ 25.,  22.,  40., ..., -31., -36.,   1.],
         [ 12.,  37.,  53., ..., -27., -37., -21.]]]], dtype=float32)

In [21]:
blob.shape

(1, 3, 300, 300)

In [22]:
net.setInput(blob)
detections=net.forward()

In [23]:
detections

array([[[[0.        , 1.        , 0.46341425, ..., 0.19970019,
          0.12055409, 0.3057317 ],
         [0.        , 1.        , 0.14459448, ..., 0.20976831,
          0.9071104 , 0.30240732],
         [0.        , 1.        , 0.13943075, ..., 0.2049298 ,
          0.67213815, 0.3031043 ],
         ...,
         [0.        , 1.        , 0.11257168, ..., 0.25742453,
          0.8292086 , 0.28682476],
         [0.        , 1.        , 0.1125468 , ..., 0.3289857 ,
          0.38267565, 0.4170944 ],
         [0.        , 1.        , 0.1124928 , ..., 0.2345663 ,
          0.55743015, 0.2670212 ]]]], dtype=float32)

In [24]:
#loop over the detections
for i in range(0,detections.shape[2]):
    confidence=detections[0,0,i,2]
    
    
    if confidence>0.5:
        #we need the X,Y coordinates
        box=detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY)=box.astype('int')
        
        #ensure the bounding boxes fall within the dimensions of the frame
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX), min(h-1,endY))
        
        
        #extract the face ROI, convert it from BGR to RGB channel, resize it to 224,224 and preprocess it
        face=image[startY:endY, startX:endX]
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face=cv2.resize(face,(224,224))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face,axis=0)
        
        (mask,withoutMask)=model.predict(face)[0]
        
        #determine the class label and color we will use to draw the bounding box and text
        label='Mask' if mask>withoutMask else 'No Mask'
        color=(0,255,0) if label=='Mask' else (0,0,255)
        
        #display the label and bounding boxes
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,1,color,3)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
        



In [ ]:
image=imutils.resize(image,width=600) 
cv2.imshow("OutPut",image)
cv2.waitKey(0)
cv2.destroyAllWindows()